### 3.4 Clustering

Python program to evaluate performance using clustering on the datasets obtained in Task 3.1.

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans

%load_ext autoreload
%autoreload 2

import bamboo.gwp_pipeline as gwp
import bamboo.star_pipeline as star
from bamboo.processing import read_csv, split_features_and_target, get_feature_and_target_names
from bamboo.analysis import *
from bamboo.selection import normalize_data

SEED = 42

GWP_CSV = 'datasets/gwp_assessment.csv'
STAR_CSV = 'datasets/star_assessment.csv'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Dataset Preperation

In [6]:
# Load and process GWP from csv
gwp_data = read_csv(GWP_CSV)
gwp_dataset = gwp.gwp_pipeline(gwp_data)

X_gwp, y_gwp = split_features_and_target(gwp_dataset)
X_gwp_names, y_gwp_name = get_feature_and_target_names(gwp_dataset)
X_gwp_train, X_gwp_test, y_gwp_train, y_gwp_test = train_test_split(X_gwp, y_gwp, test_size=0.2, random_state=SEED)
X_gwp_train, X_gwp_test = normalize_data(X_gwp_train, X_gwp_test)

# Load and process Star from csv
star_data = read_csv(STAR_CSV)
star_dataset = star.star_pipeline(star_data)

X_star, y_star = split_features_and_target(star_dataset)
X_star_names, y_star_name = get_feature_and_target_names(star_dataset)
X_star_train, X_star_test, y_star_train, y_star_test = train_test_split(X_star, y_star, test_size=0.2, random_state=SEED)
X_star_train, X_star_test = normalize_data(X_star_train, X_star_test)

## K-Means Clustering

In [ ]:
# Define the number of clusters
num_clusters = 3

# Perform kmeans clustering
kmeans_gwp = KMeans(n_clusters=num_clusters, random_state=SEED).fit(X_gwp_train)
kmeans_star = KMeans(n_clusters=num_clusters, random_state=SEED).fit(X_star_train)


#### Markdown Question: 
**Question**: Between clustering and classification algorithms, which one performed better for star dataset. Justify your answer.